In [7]:
# !pip install transformers datasets

In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from torch.utils.data import DataLoader

In [54]:
sentiment = load_dataset("yelp_polarity")
tr = sentiment['train']
ts = sentiment['test']
tr = tr.train_test_split(test_size=0.9)['train']
ts = ts.train_test_split(test_size=0.9)['train']

In [55]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2', num_labels=2) 
tokenizer.pad_token = tokenizer.eos_token

In [56]:
batch_size = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [57]:
def tokenize_function(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512, return_tensors="pt")

In [58]:
tr = tr.map(tokenize_function, batched=True)
tr.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map: 100%|███████████████████████| 56000/56000 [00:21<00:00, 2546.62 examples/s]


In [59]:
train_loader = DataLoader(tr, batch_size=batch_size, shuffle=True)

In [60]:
ts = ts.map(tokenize_function, batched=True)
ts.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
ts_loader = DataLoader(ts, batch_size=batch_size, shuffle=False)

Map: 100%|█████████████████████████| 3800/3800 [00:01<00:00, 2775.28 examples/s]


In [61]:
optimizer = AdamW(model.parameters(), lr=2e-5)

/opt/anaconda3/envs/compsci371/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [62]:
for epoch in range(3): 
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

ValueError: Expected input batch_size (4088) to match target batch_size (7).